In [ ]:

all_files = [
    '107f24d6e9_F1BE1D4184INSPIRE', '11cdce7802_B6A62F8BE0INSPIRE', '12fa5e614f_53197F206FOPENPIPELINE', '130a76ebe1_68B40B480AOPENPIPELINE', 
    '1476907971_CHADGRISMOPENPIPELINE', '1553541487_APIGENERATED', '1553541585_APIGENERATED', '1553627230_APIGENERATED', '15efe45820_D95DF0B1F4INSPIRE', 
    '1726eb08ef_60693DB04DINSPIRE', '1d056881e8_29FEA32BC7INSPIRE', '1d4fbe33f3_F1BE1D4184INSPIRE', '1df70e7340_4413A67E91INSPIRE', '2552eb56dd_2AABB46C86OPENPIPELINE', 
    '25f1c24f30_EB81FE6E2BOPENPIPELINE', '2ef3a4994a_0CCD105428INSPIRE', '2ef883f08d_F317F9C1DFOPENPIPELINE', '34fbf7c2bd_E8AD935CEDINSPIRE', 
    '3502e187b2_23071E4605OPENPIPELINE', '39e77bedd0_729FB913CDOPENPIPELINE', '420d6b69b8_84B52814D2OPENPIPELINE', '520947aa07_8FCB044F58OPENPIPELINE', 
    '551063e3c5_8FCB044F58INSPIRE', '57426ebe1e_84B52814D2OPENPIPELINE', '5fa39d6378_DB9FF730D9OPENPIPELINE', '6f93b9026b_F1BFB8B17DOPENPIPELINE', 
    '7008b80b00_FF24A4975DINSPIRE', '74d7796531_EB81FE6E2BOPENPIPELINE', '7c719dfcc0_310490364FINSPIRE', '84410645db_8D20F02042OPENPIPELINE', 
    '8710b98ea0_06E6522D6DINSPIRE', '888432f840_80E7FD39EBINSPIRE', '9170479165_625EDFBAB6OPENPIPELINE', 'a1af86939f_F1BE1D4184OPENPIPELINE', 
    'b61673f780_4413A67E91INSPIRE', 'b705d0cc9c_E5F5E0E316OPENPIPELINE', 'b771104de5_7E02A41EBEOPENPIPELINE', 'c2e8370ca3_3340CAC7AEOPENPIPELINE', 
    'c37dbfae2f_84B52814D2OPENPIPELINE', 'c644f91210_27E21B7F30OPENPIPELINE', 'c6d131e346_536DE05ED2OPENPIPELINE', 'c8a7031e5f_32156F5DC2INSPIRE', 
    'cc4b443c7d_A9CBEF2C97INSPIRE', 'd06b2c67d2_2A62B67B52OPENPIPELINE', 'd9161f7e18_C05BA1BC72OPENPIPELINE', 'dabec5e872_E8AD935CEDINSPIRE', 
    'e87da4ebdb_29FEA32BC7INSPIRE', 'ebffe540d0_7BA042D858OPENPIPELINE', 'ec09336a6f_06BA0AF311OPENPIPELINE', 
    'f0747ed88d_E74C0DD8FDOPENPIPELINE', 'f4dd768188_NOLANOPENPIPELINE', 'f56b6b2232_2A62B67B52OPENPIPELINE', 
    'f971256246_MIKEINSPIRE', 'f9f43e5144_1DB9E6F68BINSPIRE', 'fc5837dcf8_7CD52BE09EINSPIRE'
]


val_files = [
    "c644f91210_27E21B7F30OPENPIPELINE",
    "f9f43e5144_1DB9E6F68BINSPIRE",
    "1d056881e8_29FEA32BC7INSPIRE",
    "3502e187b2_23071E4605OPENPIPELINE",
    "d9161f7e18_C05BA1BC72OPENPIPELINE",
    "c8a7031e5f_32156F5DC2INSPIRE",
    "551063e3c5_8FCB044F58INSPIRE",
    "fc5837dcf8_7CD52BE09EINSPIRE",
    "39e77bedd0_729FB913CDOPENPIPELINE",
]

test_files = [
    "25f1c24f30_EB81FE6E2BOPENPIPELINE",
    "1d4fbe33f3_F1BE1D4184INSPIRE",
    "15efe45820_D95DF0B1F4INSPIRE",
    "c6d131e346_536DE05ED2OPENPIPELINE",
    "12fa5e614f_53197F206FOPENPIPELINE",
    "5fa39d6378_DB9FF730D9OPENPIPELINE",
    "ebffe540d0_7BA042D858OPENPIPELINE",
    "8710b98ea0_06E6522D6DINSPIRE",
    "84410645db_8D20F02042OPENPIPELINE",
    "a1af86939f_F1BE1D4184OPENPIPELINE"
]



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


NUM_CLASSES = 6
class_cols = ['0: Building', '1: Clutter', '2: Vegetation', '3: Water', '4: Background', '5: Car']
CLASS_NAMES = ['Building', 'Clutter', 'Vegetation', 'Water', 'Background', 'Car']
COLOR_TO_CLASS = {
    (230, 25, 75): 0,
    (145, 30, 180): 1,
    (60, 180, 75): 2,
    (245, 130, 48): 3,
    (255, 255, 255): 4,
    (0, 130, 200): 5,
    (255, 0, 255): 6
}
CLASS_TO_COLOR = {v: k for k, v in COLOR_TO_CLASS.items() if v < 6}

def plot_class_distribution_from_df(dataframe, title="Training Class Distribution"):
    pixel_sums = dataframe[class_cols].sum()
    pixel_props = pixel_sums / pixel_sums.sum()
    class_labels = [f"{i}: {CLASS_NAMES[i]}" for i in range(NUM_CLASSES)]
    colours = [np.array(CLASS_TO_COLOR[i]) / 255.0 for i in range(NUM_CLASSES)]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, pixel_props, color=colours, edgecolor='black')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Proportion")
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    for bar, prop in zip(bars, pixel_props):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{prop:.2%}",
                 ha='center', va='bottom', fontsize=9)
    plt.tight_layout()
    plt.show()

def csv_to_df(split, chip_size, ignore_threshold=0.95):
    metadata_path = "/content/chipped_data/content/chipped_data/train_metadata.csv"
    df = pd.read_csv(metadata_path)

    target_subset = 0.25
    weights = {
        "0: Building": 6.0,
        "2: Vegetation": 1.0,
        "3: Water": 18.0,
    }
    targets = {
        "0: Building": 0.55,
        "2: Vegetation": 0.05,
        "3: Water": 0.85,
    }

    if split == 'train':
        excluded_prefixes = val_files + test_files
        df = df[~df['tile_id'].apply(lambda tid: any(tid.startswith(p) for p in excluded_prefixes))].copy()

        df['total'] = df[class_cols].sum(axis=1)
        for col in class_cols:
            df[col + '_norm'] = df[col] / df['total']

        print("\n📊 Normalised Class Distribution Stats (per chip):\n")
        for col in class_cols:
            norm_col = col + '_norm'
            values = df[norm_col]
            print(f"{col}:")
            print(f"  Mean     = {values.mean():.4f}")
            print(f"  Std Dev  = {values.std():.4f}")
            print(f"  Min      = {values.min():.4f}")
            print(f"  Median   = {values.median():.4f}")
            print(f"  Max      = {values.max():.4f}\n")

        def compute_chip_score(row):
            car_ratio = row["5: Car_norm"]
            background_ratio = row["4: Background_norm"]
            if car_ratio > 0:
                return -1
            if background_ratio >= 0.70:
                return float('inf')
            return sum(weights[k] * max(targets[k] - row[f"{k}_norm"], 0) for k in targets)

        df["score"] = df.apply(compute_chip_score, axis=1)

        # Breakdown stats
        car_chips = df[df["score"] == -1]
        skipped_chips = df[df["score"] == float('inf')]
        print(f"🚗 Chips with cars (kept): {len(car_chips)} ({len(car_chips)/len(df):.2%})")
        print(f"🧱 Chips skipped due to background: {len(skipped_chips)} ({len(skipped_chips)/len(df):.2%})")

        # Example chips
        print("\n🔍 Sample Chip Distributions and Scores:")
        example = df[df["score"].notnull() & (df["score"] != float('inf'))].sample(50)
        for _, row in example.iterrows():
            print({col: round(row[col + '_norm'], 3) for col in class_cols}, f"-> Score: {row['score']:.4f}")

        # Percentiles
        filtered = df[df["score"].notnull() & (df["score"] != float('inf')) & (df["score"] != -1)]["score"]
        print("\n📉 Score Percentiles:")
        for p in [1] + list(range(5, 101, 5)):
            val = np.percentile(filtered, p)
            print(f"{p:>3}%: {val:.4f}")

        # Select chips
        n_chips = int(len(df) * target_subset)
        always_keep = df[df['score'] == -1].copy()
        rest = df[df['score'] != -1].copy()
        rest = rest[rest['score'] != float('inf')].sort_values('score')

        best_chips = pd.concat([always_keep, rest.head(n_chips - len(always_keep))])
        final_n = len(best_chips)

        # Final distribution summary
        subset = best_chips[class_cols].sum()
        actual_distribution = subset / subset.sum()
        summary_df = pd.DataFrame({
            'Class': [col.split(":")[1].strip() for col in class_cols],
            'Target': [targets.get(col, 0) for col in class_cols],
            'Actual': actual_distribution.values
        })
        summary_df['Difference'] = summary_df['Actual'] - summary_df['Target']
        from IPython.display import display
        display(summary_df)
        plot_class_distribution_from_df(best_chips, title="Training Class Distribution")
        print(f"\n📦 Selected {final_n:,} chips from {len(df):,} total ({final_n / len(df):.2%})")

        return best_chips

    elif split in ['val', 'test']:
        file_list = val_files if split == 'val' else test_files
        df = df[df['tile_id'].apply(lambda tid: any(tid.startswith(p) for p in file_list))].copy()
        df = df[(df['x'] % 256 == 0) & (df['y'] % 256 == 0)].copy()
        return df

    else:
        raise ValueError(f"Invalid split: {split}. Choose from 'train', 'val', or 'test'.")


def plot_class_distribution_from_df(dataframe, title="Training Class Distribution"):
    pixel_sums = dataframe[class_cols].sum()
    pixel_props = pixel_sums / pixel_sums.sum()

    class_labels = [f"{i}: {CLASS_NAMES[i]}" for i in range(NUM_CLASSES)]
    colours = [np.array(CLASS_TO_COLOR[i]) / 255.0 for i in range(NUM_CLASSES)]

    plt.figure(figsize=(10, 5))
    bars = plt.bar(class_labels, pixel_props, color=colours, edgecolor='black')
    plt.title(title)
    plt.xlabel("Class")
    plt.ylabel("Proportion")
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)

    for bar, prop in zip(bars, pixel_props):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f"{prop:.2%}",
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()
